In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sksurv.util import Surv
from sksurv.metrics import cumulative_dynamic_auc
from lifelines import WeibullFitter, ExponentialFitter, LogNormalFitter, LogLogisticFitter

In [63]:
data = pd.read_csv("data_ready_45.csv")

In [64]:
data["time_frame"] = data["time_frame"] + 1

In [65]:
train, test = train_test_split(data, test_size=0.2, stratify=data["GRF_STAT_PA"], random_state=42)

In [66]:
# Instantiate each fitter
wb = WeibullFitter()
ex = ExponentialFitter()
log = LogNormalFitter()
loglogis = LogLogisticFitter()# Fit to data
for model in [wb, ex, log, loglogis]:
    model.fit(durations = train["time_frame"], event_observed = train["GRF_STAT_PA"])# Print AIC
    print("The AIC value for", model.__class__.__name__, "is",  model.AIC_)

The AIC value for WeibullFitter is 80735.50265003768
The AIC value for ExponentialFitter is 80733.50500371843
The AIC value for LogNormalFitter is 81672.9421370727
The AIC value for LogLogisticFitter is 80946.41811406214


In [67]:
from lifelines import WeibullAFTFitter, LogLogisticAFTFitter, LogNormalAFTFitter

weibull_aft = LogNormalAFTFitter()
weibull_aft.fit(train, duration_col='time_frame', event_col='GRF_STAT_PA')
weibull_aft.print_summary(3)

c:\Users\Omar\AppData\Local\Programs\Python\Python311\Lib\site-packages\lifelines\fitters\__init__.py:2097: StatisticalWarning: The diagonal of the variance_matrix_ has negative values. This could be a problem with LogNormalAFTFitter's fit to the data.

It's advisable to not trust the variances reported, and to be suspicious of the fitted parameters too.

  warnings.warn(warning_text, exceptions.StatisticalWarning)


<lifelines.LogNormalAFTFitter: fitted with 16908 total observations, 12979 right-censored observations>
             duration col = 'time_frame'
                event col = 'GRF_STAT_PA'
   number of observations = 16908
number of events observed = 3929
           log-likelihood = -40166.423
         time fit was run = 2024-10-16 12:48:38 UTC

---
                             coef exp(coef)    se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param  covariate                                                                                                               
mu_    ABO_MAT             -0.222     0.801       0.073          -0.365          -0.079               0.694               0.924
       AGE                  0.029     1.029       0.003           0.023           0.035               1.024               1.035
       AGE_DON             -0.016     0.984       0.003          -0.021          -0.011               0.979               0.989
       AMIS                -0.017     0.984         NaN             NaN             NaN                 NaN                 NaN
       AMYLASE              0.000     1.000       0.000          -0.000           0.001               1.000               1.001
       ANTIHYPE_DON        -0.039     0.962       0.051          -0.138           0.060               0.871               1.062
       ART_RECON_is2        0.027     1.028       0.076          -0.121           0.175               0.886               1.192
       BLOOD_INF_CONF_DON  -0.231     0.794       0.111          -0.449          -0.013               0.639               0.988
       BMIS                 0.027     1.027         NaN             NaN             NaN                 NaN                 NaN
       BMI_CALC            -0.034     0.967       0.049          -0.130           0.062               0.878               1.064
       BMI_DON_CALC         0.011     1.011         NaN             NaN             NaN                 NaN                 NaN
       BUN_DON             -0.003     0.997       0.003          -0.009           0.003               0.991               1.003
       CARDARREST_NEURO     0.075     1.078       0.117          -0.154           0.303               0.857               1.354
       CLIN_INFECT_DON      0.116     1.123       0.071          -0.024           0.256               0.976               1.291
       CMV_DON             -0.077     0.926       0.042          -0.159           0.005               0.853               1.006
       CMV_IGG              0.011     1.011       0.085          -0.156           0.179               0.855               1.196
       CMV_IGM              0.031     1.032       0.131          -0.227           0.289               0.797               1.335
       CMV_STATUS           0.001     1.001       0.073          -0.142           0.143               0.868               1.154
       CREAT_DON           -0.095     0.909       0.068          -0.229           0.039               0.795               1.039
       CREAT_TRR           -0.009     0.991       0.008          -0.025           0.006               0.975               1.006
       DAYSWAIT_CHRON_PA   -0.000     1.000       0.000          -0.000           0.000               1.000               1.000
       DDAVP_DON            0.088     1.092       0.052          -0.013           0.189               0.987               1.208
       DIAB                 0.012     1.012       0.251          -0.479           0.503               0.619               1.654
       DIAG_PA_is5001       0.235     1.264       0.064           0.109           0.360               1.115               1.434
       DIAL_TRR            -0.068     0.935       0.060          -0.186           0.050               0.831               1.051
       DRMIS               -0.026     0.974         NaN             NaN             NaN                 NaN                 NaN
       DUCT_MGMT_1          2.842    17.147 1347464.891    -264097

In [68]:
y_train = train[["time_frame", "GRF_STAT_PA"]]
y_real_train = Surv.from_dataframe("GRF_STAT_PA", "time_frame", y_train)


In [69]:
y = test[["time_frame", "GRF_STAT_PA"]]

x = test.drop(columns=["time_frame", "GRF_STAT_PA"], axis=1)

In [70]:
y_real = Surv.from_dataframe("GRF_STAT_PA", "time_frame", y)

time_points = [30, 365, 365*2, 365*5]

preds = weibull_aft.predict_cumulative_hazard(df=x, times=time_points)

auc, mean_auc = cumulative_dynamic_auc(y_real_train, y_real, preds.T, time_points)

In [71]:
print("Dynamic AUC values at different time points:")
for t, auc in zip(time_points, auc):
    print(f"Time {t:.2f}: AUC = {auc:.3f}")

print(f"\nMean Dynamic AUC: {mean_auc:.3f}")

Dynamic AUC values at different time points:
Time 30.00: AUC = 0.768
Time 365.00: AUC = 0.716
Time 730.00: AUC = 0.742
Time 1825.00: AUC = 0.718

Mean Dynamic AUC: 0.726
